In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_pickle('startingdata_filename_date_currency_loanamount.dat')

Convert the loan amounts in US dollars.

In [3]:
currency_set = set(data['Currency'].values)
print(currency_set)

{'eur', 'jpy', 'usd', 'us', 'gbp', 'uss', 'dem', 'y', '$', None, 'c', '€', '£', '¥', 'sdr', 'e', 'egp', 'frf', 's'}


In [4]:
exchangerate = pd.read_csv('../exchangerates.csv')

In [5]:
#cut out the historical data of the exchange rates for 
#euro, dem, yen, gbp, frf
euro= exchangerate.loc[exchangerate['LOCATION'] == 'EU27_2020']
dem = exchangerate.loc[exchangerate['LOCATION'] == 'DEU']
yen = exchangerate.loc[exchangerate['LOCATION'] == 'JPN']
gbp = exchangerate.loc[exchangerate['LOCATION'] == 'GBR']
frf = exchangerate.loc[exchangerate['LOCATION'] == 'FRA']

In [6]:
#create a list of the loan amounts coverted to usd
#call it to_usd
#for each loan, find the year & the currency 
#from 'Extracted_Information.dat',
#

to_usd = []
for i in range(len(data)):
    amount = 0
    if data['Currency'].iloc[i] == 'usd' or data['Currency'].iloc[i] == '$' or data['Currency'].iloc[i] == 'sdr':
        amount = data['Amount_loan'].iloc[i]
    elif data.iloc[i,3] == 'eur' or data.iloc[i,3] == 'e' or data.iloc[i,3] == '€':
        yearnumber = data.iloc[i,1].year
        for j in range(len(euro)):
            if euro['TIME'].iloc[j] == yearnumber:
                value = euro['Value'].iloc[j]
                break
        amount = data['Amount_loan'].iloc[i]*value
    elif data.iloc[i,3] == 'dem':
        yearnumber = data.iloc[i,1].year
        for d in range(len(dem)):
            if dem['TIME'].iloc[d] == yearnumber:
                value = dem['Value'].iloc[d]
                break
        amount = data['Amount_loan'].iloc[i]*value
    elif data.iloc[i,3] == 'jpy':
        yearnumber = data.iloc[i,1].year
        for y in range(len(yen)):
            if yen['TIME'].iloc[y] == yearnumber:
                value = yen['Value'].iloc[y]
                break
        amount = data1['Amount_loan'].iloc[i]*value
    elif data.iloc[i,3] == 'gbp' or data.iloc[i,3] == '£':
        yearnumber = data1.iloc[i,1].year
        for g in range(len(gbp)):
            if gbp['TIME'].iloc[g] == yearnumber:
                value = gbp['Value'].iloc[g]
                break
        amount = data['Amount_loan'].iloc[i]*value
    elif data.iloc[i,3] == 'frf':
        yearnumber = data1.iloc[i,1].year
        for f in range(len(frf)):
            if frf['TIME'].iloc[f] == yearnumber:
                value = frf['Value'].iloc[f]
                break
        amount = data['Amount_loan'].iloc[i]*value
        
    to_usd.append(int(amount))

In [7]:
data["Amount_in_usd"] = pd.Series(to_usd)

In [8]:
for i in data.index:
    if data["Amount_in_usd"].iloc[i] == 0:
        data["Amount_in_usd"].iloc[i] = None

/opt/homebrew/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Read the text files extracted by 'pdfminer' and 'pytesseract' and combine them in one dataframe.
Call the combined dataframe by 'combined' whose columns are 'filename' and 'extracted_text'.

In [9]:
pdfminer = pd.read_pickle('pdfminer_ables.dat')
pytess = pd.read_pickle('pytesseract_results.dat')
pdfminer['extracted_text'] = pdfminer['de_headed']
pytess['extracted_text'] = pytess['reparse']
combined = pytess.combine_first(pdfminer)
combined = combined[['filename','extracted_text']]

Define two functions called 'spacing_and_lower' and 'read_address'. 'spacing_and_lower' of a string returns the string replaced all spaces by no space and makes the all upper charaters of the given string into lowercase. 'read_address' extracts the part containing the address of the borrower given a text file. 

In [10]:
def spacing_and_lower(docu):
    docu = docu.replace('  ',' ')
    docu = docu.replace('   ',' ')
    docu = docu.lower()
    return docu

We search a string 'addresses' and then look for some starting point and end point to cut off the part containing the address of the borower. For example, the addresses appears after a substring 'For the borrower:' and before 'For the bank'.

In [11]:
def read_address(docu, pin0= 'addresses', pin1 = 'start', pin2 = 'end'):
    start = docu.find(pin0)
    borrower = docu[start:].find(pin1)
    colon = docu[start+borrower:].find(':')
    bank = docu[start+borrower+colon:].find(pin2)
    add = docu[start+borrower+colon+1:start+borrower+colon+bank]
    if 'in witness' in add:
        end2 = docu[start+borrower+colon+1:].find('in witness')
        add = add[:end2]
    return add   

In [12]:
address1 = []
for i in range(len(combined)):
    docu = combined.iloc[i,1]
    docu = spacing_and_lower(docu)
    address = read_address(docu, pin0 = 'addresses', pin1 = 'for the borrower', pin2 = 'for the bank')
    if len(address) < 8:
        address = read_address(docu, pin0 = 'addresses', pin1 = 'for the borrower', pin2 = 'international bank')
    address1.append(address)

In [13]:
address2 = []
for i in range(len(combined)):
    docu = combined.iloc[i,1]
    docu = spacing_and_lower(docu)
    address = read_address(docu, pin0 = 'addresses', pin1 = 'address is', pin2 = 'the bank')
    address2.append(address)


In [14]:
address3 = []
for i in range(len(combined)):
    docu = combined.iloc[i,1]
    docu = spacing_and_lower(docu)
    address = read_address(docu, pin0 = 'addresses', pin1 = 'the borrower', pin2 = 'the world bank')
    address3.append(address)

In [15]:
for i in range(len(address1)):
    if len(address1[i]) < 8 & len(address2[i]) > 8:
        address1[i] = address2[i]
    elif len(address1[i]) > 500:
        address1[i] = address2[i]

In [16]:
for i in range(len(address1)):
    if len(address1[i]) < 8 & len(address3[i]) > 8:
        address1[i] = address3[i]

In [17]:
data = pd.read_pickle('startingdata_filename_date_currency_loanamount.dat')

In [18]:
data['Address'] = pd.Series(address1)

In [19]:
a = 0
for i in range(len(address1)):
    if len(address1[i]) < 8:
        data['Address'] = None
        a = a+1

In [20]:
data['Address'] = pd.Series(address1)

In [22]:
data.to_pickle('data_after_extracting_address.dat')

There are 113 missing values in 'Address'.

In [23]:
a

113

Now we extract the country codes from 'Address'. I edited my code after comparing the country codes extracted from the cover page.

In [24]:
worldmap = pd.read_csv('../world_bank_country_mappings.csv')

In [25]:
country_name = []
country_code = []
income_level = []
region = []
for i in range(len(data)):
    address = data["Address"].iloc[i]
    address = address.replace('\n','')
    address = address.replace(' ','')
    address = address.replace('  ','')
    address = address.replace("’","")
    address = address.lower()
    for j in range(len(worldmap['name'])):
        
        if worldmap['name'][j] == 'Africa':
            continue
            
        if worldmap['name'][j] == 'United States':
            continue
            
        if worldmap['name'][j] == 'World':
            continue
            
        if worldmap['name'][j] == 'Niger' and address.find('nigeria') > -1:
            continue
            
        if worldmap['name'][j] == 'Oman' and address.find('romania') > -1:
            continue
        
        if worldmap ['name'][j] == 'Oman' and address.find('phi') > -1:
            continue
            
        if worldmap ['name'][j] == 'Mali' and address.find('turkey') > -1:
            continue
        
        if worldmap['name'][j] == 'Guinea' and address.find('papua') > -1:
            countryname = 'Papua New Guinea'
            countrycode = 'PNG'
            incomelevel = 'Lower middle income'
            regi = 'East Asia & Pacific'
            break
        
        if worldmap['name'][j] == 'Spain' and address.find('trinidad')> -1:
            countryname = 'Trinidad and Tobago'
            countrycode = 'TTO'
            incomelevel = 'High income'
            regi = 'Latin America & Caribbean '
            break
        
        if worldmap['name'][j] == 'Chile' and address.find('paraguay')> -1:
            countryname = 'Paraguay'
            countrycode = 'PRY'
            incomelevel = 'Upper middle income'
            regi = 'Latin America & Caribbean '
            break
        
        else:    
            name = worldmap['name'][j]
            name = name.replace(' ','')
            name = name.replace("'","")
            name = name.replace('  ','')
            name = name.lower()
            cut = name.find(',')
            if cut > 0:
                name = name[:cut]
            countryname = ''
            countrycode = ''
            incomelevel = ''
            regi = ''
            
            
                
            if address.find(name) > -1:
                if name == 'congo' and address.find('brazzaville')>-1:
                    countryname = 'Congo,Rep.'
                    countrycode = 'COG'
                    incomelevel = 'Lower middle income'
                    regi = 'Sub-Saharan Africa'
                    break
                    
                else:  
                    countryname = worldmap['name'][j]
                    countrycode = worldmap['id'][j]
                    incomelevel = worldmap['incomeLevel.value'][j]
                    regi = worldmap['region.value'][j]
                    break            
            
            
            city = worldmap['capitalCity'][j]
            if type(city) == str:
                city = city.replace(' ','')
                city = city.replace("'","")
                city = city.replace('  ','')
                city = city.lower()
                if address.find(name) == -1 and address.find(city) >-1:
                    countryname = worldmap['name'][j]
                    countrycode = worldmap['id'][j]
                    incomelevel = worldmap['incomeLevel.value'][j]
                    regi = worldmap['region.value'][j]
                    break                       
                    
                    
    if countryname == '':
        if address.find('méxico') > -1 or address.find('mexican') > -1:
            countryname = 'Mexico'
            countrycode = 'MEX'
            incomelevel = 'Upper middle income'
            regi = 'Latin America & Caribbean '
    
    if address.find('yugoslavia') > -1:
            countryname = 'Yugoslavia'
            countrycode = 'YUGOS'
            incomelevel = 'Upper middle income'
            regi = 'Europe & Central Asia'
    
    if countryname == '':
        countryname = None
        countrycode = None
        incomelevel = None
        regi = None
        
        
            
    country_name.append(countryname)
    country_code.append(countrycode)
    income_level.append(incomelevel)
    region.append(regi)

In [26]:
data['Address_to_countrycode'] = pd.Series(country_code)
data['Country_name'] = pd.Series(country_name)
data['Income_level'] = pd.Series(income_level)
data['Region'] = pd.Series(region)

In [27]:
a = 0
for i in data.index:
    if data['Address_to_countrycode'][i] == None:
        a = a+1

In [28]:
a

201

In [29]:
code_from_cover = pd.read_csv('Includes_Country_Name.csv')

In [30]:
cover_code = code_from_cover["Country_Code"]
address_code = data["Address_to_countrycode"]

In [31]:
same = []
nan = []
none = []
twonulls = []
twodiff = []


for i in range(3205):
    if cover_code[i] != address_code[i] and type(cover_code[i])== float:
        if address_code[i] == None:
            twonulls.append(i)
        else:
            nan.append(i)
    elif cover_code[i] != address_code[i] and type(cover_code[i])== str:
        if address_code[i] == None:
            none.append(i)
        else:
            twodiff.append(i)
    else:
        same.append(i)

In [32]:
print(len(same),len(nan),len(none),len(twonulls),len(twodiff))

2460 425 136 65 119


In [33]:
bettercode = []
for i in twodiff:
    address = data["Address"][i]
    address = address.replace('\n','')
    address = address.replace(' ','')
    address = address.replace('  ','')
    address = address.replace("’","")
    address = address.lower()
    
    if address_code[i] == 'BRA' and address.find('brazil')> -1:
        bettercode.append(i)
        continue
        
    if address_code[i] != 'YUGOS':
    
        ind = int(worldmap[worldmap['id'] == address_code[i]].index.values)
    
        country = worldmap['name'][ind]
        country = country.lower()
        
        if address.find(country) >-1:
            bettercode.append(i)
            continue
                
        city = worldmap['capitalCity'][ind]
    
        if type(city) == str:
            city = city.replace(' ','')
            city = city.replace("'","")
            city = city.replace('  ','')
            city = city.lower()
            if address.find(city) >-1:
                bettercode.append(i)

In [34]:
len(bettercode)

105

In [35]:
rest_twodiff = list(set(twodiff) - set(bettercode))

In [36]:
for i in rest_twodiff:
    if address_code[i] == 'Yugoslavia':
        continue
    if cover_code[i] == 'THA' or cover_code[i] == 'ALB':
        address_code[i] = cover_code[i]       

In [37]:
for i in none:
    address_code[i] = cover_code[i]

In [38]:
a = 0
for i in data.index:
    if address_code[i] == None:
        a = a+1

In [39]:
a

65

In [40]:
data.to_pickle('finaldata_after_editting_countrycode.dat')